# Import & Load Inmigration data

### Config

In [ ]:
import sys
import configparser
config = configparser.ConfigParser()
config.read_file(open('../../../settings.ini'))

sys.path.insert(0, config.get('PATHS','libs_path'))
engine_path = config.get('DATABASE','engine_path')

### Main params

In [ ]:
remote_path = 'http://pacha.datawheel.us/datachile/immigration/visas/3_clean_datasets/'
local_path = '../data/'

In [ ]:
import postgres #from local file postgres.py
from commons import inline_table_xml, download_file, download_zip_file, extract_zip_file #from local file commons.py

import json
import pandas as pd
from sqlalchemy import create_engine

### Download & unzip file

In [ ]:
#Download and unzip
download_zip_file(remote_path,local_path,'visas.zip')
extract_zip_file(local_path,'temp.zip')

### Load file

In [ ]:
#Open unzipped file
df = pd.read_csv(local_path+'3_clean_datasets/visas.csv',delimiter=",")
df = df[['year','comuna_datachile_id','sex_id','activity_id','birth_date','age','studies_id','country_code','visa_type_id']]
df = df.astype({'year':'int','comuna_datachile_id':'int','sex_id':'int','activity_id':'int','studies_id':'int','age':'int','country_code':'int','visa_type_id':'int'})
list(df)

### Ingest

In [ ]:
engine = create_engine(engine_path)
db = postgres.PostgresDriver(engine)
db.to_sql(df, 'demographics', 'fact_immigration_records')

### Add Indices to foreign keys

In [ ]:
engine.execute("""
CREATE INDEX fact_immigration_records_comuna_datachile_id_index 
ON demographics.fact_immigration_records (comuna_datachile_id)
""")

engine.execute("""
CREATE INDEX fact_immigration_records_year_index 
ON demographics.fact_immigration_records ("year");
""")



### Relation tables

```xml
<InlineTable alias="sex">
  <ColumnDefs>
    <ColumnDef name="id" type="Numeric" />
    <ColumnDef name="description" type="String" />
    <ColumnDef name="es_description" type="String" />
  </ColumnDefs>
  <Rows>
    <Row>
      <Value column="id">0</Value>
      <Value column="description">No informa</Value>
      <Value column="es_description">No informa</Value>
    </Row>
  <Row>
      <Value column="id">1</Value>
      <Value column="description">Women</Value>
      <Value column="es_description">Mujer</Value>
    </Row>
  <Row>
      <Value column="id">2</Value>
      <Value column="description">Men</Value>
      <Value column="es_description">Hombre</Value>
    </Row>
  </Rows>
</InlineTable>
```

In [ ]:
d2 = download_file(remote_path,local_path,'activity_id.csv')
print (inline_table_xml(d2, 'activity', 'activity_id', 'activity'))

In [ ]:
d3 = download_file(remote_path,local_path,'studies_id.csv')
print (inline_table_xml(d3, 'studies', 'studies_id', 'studies'))

In [ ]:
d4 = download_file(remote_path,local_path,'visa_type.csv')
print (inline_table_xml(d4, 'visa_type', 'visa_type_id', 'visa_type'))